In [1]:
import pandas as pd
import numpy as np
import re
import os

#Webscraping/NLP
import requests
from requests import get
from bs4 import BeautifulSoup
import time
import nltk
import requests
import unicodedata
from nltk.corpus import stopwords

#Visualizations
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

#My imports
import prepare as p
import acquire as a
from importlib import reload

In [2]:
#Get the data
df = pd.read_pickle('data/michelin_df.pickle')
df.head()

,name,address,location,price,cuisine,longitude,latitude,phone_number,url,website_url,award,facilities_and_services,data
0,King's Joy,"2 Wudaoying Hutong, Beijing, China Mainland","Beijing, China Mainland",¥¥¥,Vegetarian,116.410004,39.946681,8.610840e+11,https://guide.michelin.com/en/beijing-municipa...,NaN,3 MICHELIN Stars,"Air conditioning,American Express credit card,...","Just a stone’s throw from Yonghe Temple, this ..."
1,Xin Rong Ji (Xinyuan South Road),"1F, East Tower, Genesis Beijing, 8 Xinyuan Sou...","Beijing, China Mainland",¥¥¥¥,Taizhou,116.450148,39.946380,8.610650e+11,https://guide.michelin.com/en/beijing-municipa...,NaN,3 MICHELIN Stars,"Air conditioning,Car park,China UnionPay,Count...",This branch of the chain restaurant opened in ...
2,Taian Table,"101-102, Building No. 1, Garden Office, No.161...","Shanghai, China Mainland",¥¥¥¥,Innovative,121.474049,31.221807,8.617302e+12,https://guide.michelin.com/en/shanghai-municip...,https://taian-table.cn/,3 MICHELIN Stars,"Air conditioning,American Express credit card,...",A fixture on the city’s dining scene since 201...
3,Ultraviolet by Paul Pairet,'somewhere in Shanghai' - meet at Mr & Mrs Bun...,"Shanghai, China Mainland",¥¥¥¥,Innovative,121.485090,31.240358,NaN,https://guide.michelin.com/en/shanghai-municip...,https://uvbypp.cc/,3 MICHELIN Stars,"Air conditioning,American Express credit card,...",This multi-sensory experience at a secret loca...
4,Quince,"470 Pacific Ave., San Francisco, 94133, USA","San Francisco, USA",$$$$,"Contemporary, Californian",-122.403261,37.797505,1.415776e+10,https://guide.michelin.com/en/california/san-f...,http://www.quincerestaurant.com,3 MICHELIN Stars,"Air conditioning,American Express credit card,...","An air of refinement infuses this dining room,..."


In [140]:
#TODO make visualization of distribution of Michelin star levels

In [141]:
#TODO drop phone number column?

In [3]:
# # Turn NaN values in price to nothing, so that it can be recast into the price_level column
# df['price'] = df['price'].fillna('').astype('str')

In [96]:
# #Casting a new column, price level, using length of column
# df['price_level'] = df['price'].apply(lambda x: len(x))

In [101]:
# #Setting mode of this column
# mode= df.price_level.mode()[0]
# mode

2

In [102]:
# #Imputing 0s in price level with the mode for this column
# df['price_level'] = df['price_level'].replace(0, mode)

In [104]:
#Separate City and country columns to create two new columns
df.location

0        Beijing, China Mainland
1        Beijing, China Mainland
2       Shanghai, China Mainland
3       Shanghai, China Mainland
4             San Francisco, USA
                  ...           
6775              Warsaw, Poland
6776              Cracow, Poland
6777              Cracow, Poland
6778              Warsaw, Poland
6779              Warsaw, Poland
Name: location, Length: 6780, dtype: object

In [105]:
# #splitting location columns into two columns
# df[['city', 'country']] = df['location'].str.split(',', 1, expand=True)

In [107]:
# #Dropping phone number column
# df = df.drop(['phone_number'], axis=1)

In [108]:
# #Dropping website url column
# df = df.drop(['website_url'], axis=1)

In [109]:
# #lower case all column values if column is object/string type
# df = df.apply(lambda x: x.str.lower() if(x.dtype == 'object') else x)

In [41]:
#TODO Create clean dtypes function

In [127]:
# #Turn city/states into city states, impute city into country column
# df['country'] = np.where(pd.isna(df['country']), df['city'], df['country'])

In [154]:
#For testing this function, getting clean data
df = pd.read_pickle('data/michelin_df.pickle')
df.head()

,name,address,location,price,cuisine,longitude,latitude,phone_number,url,website_url,award,facilities_and_services,data
0,King's Joy,"2 Wudaoying Hutong, Beijing, China Mainland","Beijing, China Mainland",¥¥¥,Vegetarian,116.410004,39.946681,8.610840e+11,https://guide.michelin.com/en/beijing-municipa...,NaN,3 MICHELIN Stars,"Air conditioning,American Express credit card,...","Just a stone’s throw from Yonghe Temple, this ..."
1,Xin Rong Ji (Xinyuan South Road),"1F, East Tower, Genesis Beijing, 8 Xinyuan Sou...","Beijing, China Mainland",¥¥¥¥,Taizhou,116.450148,39.946380,8.610650e+11,https://guide.michelin.com/en/beijing-municipa...,NaN,3 MICHELIN Stars,"Air conditioning,Car park,China UnionPay,Count...",This branch of the chain restaurant opened in ...
2,Taian Table,"101-102, Building No. 1, Garden Office, No.161...","Shanghai, China Mainland",¥¥¥¥,Innovative,121.474049,31.221807,8.617302e+12,https://guide.michelin.com/en/shanghai-municip...,https://taian-table.cn/,3 MICHELIN Stars,"Air conditioning,American Express credit card,...",A fixture on the city’s dining scene since 201...
3,Ultraviolet by Paul Pairet,'somewhere in Shanghai' - meet at Mr & Mrs Bun...,"Shanghai, China Mainland",¥¥¥¥,Innovative,121.485090,31.240358,NaN,https://guide.michelin.com/en/shanghai-municip...,https://uvbypp.cc/,3 MICHELIN Stars,"Air conditioning,American Express credit card,...",This multi-sensory experience at a secret loca...
4,Quince,"470 Pacific Ave., San Francisco, 94133, USA","San Francisco, USA",$$$$,"Contemporary, Californian",-122.403261,37.797505,1.415776e+10,https://guide.michelin.com/en/california/san-f...,http://www.quincerestaurant.com,3 MICHELIN Stars,"Air conditioning,American Express credit card,...","An air of refinement infuses this dining room,..."


In [150]:
# def create_features(df):
#     '''
#     This function takes in the dataframe, drops unnecessary columns,
#     and creates new columns/features for exploration and potential
#     classification purposes. It returns the dataframe with applications
#     '''
#     # Dropping unnecessary columns
#     df = df.drop(['phone_number', 'website_url'], axis=1)
    
#     #lower case all column values if column is object/string type
#     df = df.apply(lambda x: x.str.lower() if(x.dtype == 'object') else x)
    
#     # Turn NaN values in price to 'nothing', so that it can be recast into the price_level column
#     df['price'] = df['price'].fillna('').astype('str')
#     # Casting a new column, price level, using length of column
#     df['price_level'] = df['price'].apply(lambda x: len(x))
#     # impute price level "0" with the mode for this column
#     mode= df.price_level.mode()[0]
#     df['price_level'] = df['price_level'].replace(0, mode)
    
#     # splitting location columns into two columns
#     df[['city', 'country']] = df['location'].str.split(', ', 1, expand=True)
#     # Turn city/states into city states, impute city into country column (i.e Hong Kong, Hong Kong)
#     df['country'] = np.where(pd.isna(df['country']), df['city'], df['country'])

#     return df


In [4]:
#For testing this function, getting clean data
df = pd.read_pickle('data/michelin_df.pickle')
df.head()

,name,address,location,price,cuisine,longitude,latitude,phone_number,url,website_url,award,facilities_and_services,data
0,King's Joy,"2 Wudaoying Hutong, Beijing, China Mainland","Beijing, China Mainland",¥¥¥,Vegetarian,116.410004,39.946681,8.610840e+11,https://guide.michelin.com/en/beijing-municipa...,NaN,3 MICHELIN Stars,"Air conditioning,American Express credit card,...","Just a stone’s throw from Yonghe Temple, this ..."
1,Xin Rong Ji (Xinyuan South Road),"1F, East Tower, Genesis Beijing, 8 Xinyuan Sou...","Beijing, China Mainland",¥¥¥¥,Taizhou,116.450148,39.946380,8.610650e+11,https://guide.michelin.com/en/beijing-municipa...,NaN,3 MICHELIN Stars,"Air conditioning,Car park,China UnionPay,Count...",This branch of the chain restaurant opened in ...
2,Taian Table,"101-102, Building No. 1, Garden Office, No.161...","Shanghai, China Mainland",¥¥¥¥,Innovative,121.474049,31.221807,8.617302e+12,https://guide.michelin.com/en/shanghai-municip...,https://taian-table.cn/,3 MICHELIN Stars,"Air conditioning,American Express credit card,...",A fixture on the city’s dining scene since 201...
3,Ultraviolet by Paul Pairet,'somewhere in Shanghai' - meet at Mr & Mrs Bun...,"Shanghai, China Mainland",¥¥¥¥,Innovative,121.485090,31.240358,NaN,https://guide.michelin.com/en/shanghai-municip...,https://uvbypp.cc/,3 MICHELIN Stars,"Air conditioning,American Express credit card,...",This multi-sensory experience at a secret loca...
4,Quince,"470 Pacific Ave., San Francisco, 94133, USA","San Francisco, USA",$$$$,"Contemporary, Californian",-122.403261,37.797505,1.415776e+10,https://guide.michelin.com/en/california/san-f...,http://www.quincerestaurant.com,3 MICHELIN Stars,"Air conditioning,American Express credit card,...","An air of refinement infuses this dining room,..."


In [5]:
train, validate, test = p.prepare_michelin(df)

In [6]:
train.head()

,name,address,location,price,cuisine,longitude,latitude,url,award,facilities_and_services,data,price_level,city,country,clean,lemmatized
3016,rüssel's landhaus,"büdlicherbrück 1, naurath/wald, 54426, germany","naurath/wald, germany",€€€€,"creative, market cuisine",6.892007,49.768126,https://guide.michelin.com/en/rheinland-pfalz/...,1 michelin star,"american express credit card,car park,credit c...",the charm of this country house makes itself f...,4,naurath/wald,germany,charm country house makes felt approach via sm...,charm country house make felt approach via sma...
1802,iroha,"5f, 1-3-19 sonezakishinchi, kita-ku, osaka, 53...","osaka, japan",¥¥¥,japanese,135.495812,34.697842,https://guide.michelin.com/en/osaka-region/osa...,1 michelin star,"air conditioning,american express credit card,...",the charm here is in the wide selection of dis...,3,osaka,japan,charm wide selection dishes consists classic d...,charm wide selection dish consists classic dis...
91,kei,"5 rue du coq-héron, paris, 75001, france","paris, france",€€€€,modern cuisine,2.342285,48.864395,https://guide.michelin.com/en/ile-de-france/pa...,3 michelin stars,"air conditioning,american express credit card,...","""kei"" is kei kobayashi, a chef born in the jap...",4,paris,france,kei kei kobayashi born japanese city nagano tr...,kei kei kobayashi born japanese city nagano tr...
5283,famous sungei road trishaw laksa,"hong lim market and food centre, #02-66, blk 5...",singapore,$,street food,103.845742,1.285180,https://guide.michelin.com/en/singapore-region...,bib gourmand,NONE,the laksa is famous for its rich flavour. the ...,1,singapore,singapore,laksa famous rich flavour fruit juice mee siam...,laksa famous rich flavour fruit juice mee siam...
1424,magorabin,"corso san maurizio 61/d, turin, 10124, italy","turin, italy",€€€,"creative, contemporary",7.696044,45.068465,https://guide.michelin.com/en/piemonte/torino/...,1 michelin star,"air conditioning,american express credit card,...",the owner-chef at this restaurant focuses on r...,3,turin,italy,ownerchef focuses recipes firmly rooted local ...,ownerchef focus recipe firmly rooted local tra...


In [7]:
train.lemmatized.value_counts()

none                                                 4
temporarily closed                                   3
                                                     2
starting small shop mueang thong thani winning...    1
stand main square delightful village serf exce...    1
                                                    ..
one another filled brim creative impulse cold ...    1
quiet little village attractive rural setting ...    1
gastronomy tatsuhiro takayama blend passion le...    1
santa barbara public market full alluring eate...    1
part antinori estate elegant feel despite desc...    1
Name: lemmatized, Length: 3790, dtype: Int64

In [10]:
df[df.data == 'None']

,name,address,location,price,cuisine,longitude,latitude,phone_number,url,website_url,award,facilities_and_services,data
7,Manresa,"320 Village Ln., Los Gatos, 95030, USA","Los Gatos, USA",$$$$,"Contemporary, Californian",-121.980710,37.227610,1.408354e+10,https://guide.michelin.com/en/california/los-g...,https://www.manresarestaurant.com/,3 MICHELIN Stars,"Air conditioning,Credit card / Debit card acce...",None
1285,Damiano Nigro,"Borgata Cerequio 18, La Morra, 12064, Italy","La Morra, Italy",€€€,Contemporary,7.938917,44.624578,3.901735e+10,https://guide.michelin.com/en/piemonte/la-morr...,https://palascerequio.com/,1 MICHELIN Star,"Air conditioning,American Express credit card,...",None
5674,Bell'Arrivo,"rue de Nivelles 30, Ronquières, 7090, Belgium","Ronquières, Belgium",€€,Italian,4.228022,50.606367,3.267411e+09,https://guide.michelin.com/en/hainaut/ronquire...,https://www.bellarrivo.com/,Bib Gourmand,"Credit card / Debit card accepted,Mastercard c...",None
5765,Pizzeria Bebu,"1521 N. Fremont St., Chicago, 60642, USA","Chicago, USA",$$,"Pizza, American",-87.650998,41.909245,1.312281e+10,https://guide.michelin.com/en/illinois/chicago...,https://bebu.pizza/,Bib Gourmand,"Air conditioning,Brunch,Credit card / Debit ca...",None
6344,Chez Georges,"Kirchstraße 38, Achern, 77855, Germany","Achern, Germany",€,"Country cooking, Alsatian",8.074822,48.632143,4.978417e+10,https://guide.michelin.com/en/baden-wurttember...,https://www.hotel-sha.de/,Bib Gourmand,"Air conditioning,American Express credit card,...",None
6772,Più Iguatemi,"Avenida Brigadeiro Faria Lima 2232, São Paulo,...","São Paulo, Brazil",NaN,"Italian, Creative",-46.688315,-23.577106,5.511320e+11,https://guide.michelin.com/en/sao-paulo-region...,https://www.piurestaurante.com.br/,Bib Gourmand,"Air conditioning,American Express credit card,...",None


In [11]:
ex_michelin = df[df.data == 'None']
ex_michelin

,name,address,location,price,cuisine,longitude,latitude,phone_number,url,website_url,award,facilities_and_services,data
7,Manresa,"320 Village Ln., Los Gatos, 95030, USA","Los Gatos, USA",$$$$,"Contemporary, Californian",-121.980710,37.227610,1.408354e+10,https://guide.michelin.com/en/california/los-g...,https://www.manresarestaurant.com/,3 MICHELIN Stars,"Air conditioning,Credit card / Debit card acce...",None
1285,Damiano Nigro,"Borgata Cerequio 18, La Morra, 12064, Italy","La Morra, Italy",€€€,Contemporary,7.938917,44.624578,3.901735e+10,https://guide.michelin.com/en/piemonte/la-morr...,https://palascerequio.com/,1 MICHELIN Star,"Air conditioning,American Express credit card,...",None
5674,Bell'Arrivo,"rue de Nivelles 30, Ronquières, 7090, Belgium","Ronquières, Belgium",€€,Italian,4.228022,50.606367,3.267411e+09,https://guide.michelin.com/en/hainaut/ronquire...,https://www.bellarrivo.com/,Bib Gourmand,"Credit card / Debit card accepted,Mastercard c...",None
5765,Pizzeria Bebu,"1521 N. Fremont St., Chicago, 60642, USA","Chicago, USA",$$,"Pizza, American",-87.650998,41.909245,1.312281e+10,https://guide.michelin.com/en/illinois/chicago...,https://bebu.pizza/,Bib Gourmand,"Air conditioning,Brunch,Credit card / Debit ca...",None
6344,Chez Georges,"Kirchstraße 38, Achern, 77855, Germany","Achern, Germany",€,"Country cooking, Alsatian",8.074822,48.632143,4.978417e+10,https://guide.michelin.com/en/baden-wurttember...,https://www.hotel-sha.de/,Bib Gourmand,"Air conditioning,American Express credit card,...",None
6772,Più Iguatemi,"Avenida Brigadeiro Faria Lima 2232, São Paulo,...","São Paulo, Brazil",NaN,"Italian, Creative",-46.688315,-23.577106,5.511320e+11,https://guide.michelin.com/en/sao-paulo-region...,https://www.piurestaurante.com.br/,Bib Gourmand,"Air conditioning,American Express credit card,...",None


In [17]:
df = df[df.data != 'None']

,name,address,location,price,cuisine,longitude,latitude,phone_number,url,website_url,award,facilities_and_services,data
0,King's Joy,"2 Wudaoying Hutong, Beijing, China Mainland","Beijing, China Mainland",¥¥¥,Vegetarian,116.410004,39.946681,8.610840e+11,https://guide.michelin.com/en/beijing-municipa...,NaN,3 MICHELIN Stars,"Air conditioning,American Express credit card,...","Just a stone’s throw from Yonghe Temple, this ..."
1,Xin Rong Ji (Xinyuan South Road),"1F, East Tower, Genesis Beijing, 8 Xinyuan Sou...","Beijing, China Mainland",¥¥¥¥,Taizhou,116.450148,39.946380,8.610650e+11,https://guide.michelin.com/en/beijing-municipa...,NaN,3 MICHELIN Stars,"Air conditioning,Car park,China UnionPay,Count...",This branch of the chain restaurant opened in ...
2,Taian Table,"101-102, Building No. 1, Garden Office, No.161...","Shanghai, China Mainland",¥¥¥¥,Innovative,121.474049,31.221807,8.617302e+12,https://guide.michelin.com/en/shanghai-municip...,https://taian-table.cn/,3 MICHELIN Stars,"Air conditioning,American Express credit card,...",A fixture on the city’s dining scene since 201...
3,Ultraviolet by Paul Pairet,'somewhere in Shanghai' - meet at Mr & Mrs Bun...,"Shanghai, China Mainland",¥¥¥¥,Innovative,121.485090,31.240358,NaN,https://guide.michelin.com/en/shanghai-municip...,https://uvbypp.cc/,3 MICHELIN Stars,"Air conditioning,American Express credit card,...",This multi-sensory experience at a secret loca...
4,Quince,"470 Pacific Ave., San Francisco, 94133, USA","San Francisco, USA",$$$$,"Contemporary, Californian",-122.403261,37.797505,1.415776e+10,https://guide.michelin.com/en/california/san-f...,http://www.quincerestaurant.com,3 MICHELIN Stars,"Air conditioning,American Express credit card,...","An air of refinement infuses this dining room,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6775,Szóstka,"Plac Powstańców Warszawy 9, Warsaw, 00 039, Po...","Warsaw, Poland",€€,"Polish, Modern Cuisine",21.012698,52.235803,4.822470e+10,https://guide.michelin.com/en/masovia/warsaw/r...,https://warszawa.hotel.com.pl/hotel-warszawa,Bib Gourmand,"Air conditioning,American Express credit card,...",Take in rooftop views from this appealing glas...
6776,Fiorentina,"ul. Grodzka 63, Cracow, 31 044, Poland","Cracow, Poland",€€,"Creative, Polish",19.938179,50.055898,4.812426e+10,https://guide.michelin.com/en/lesser-poland/cr...,https://fiorentina.com.pl/,Bib Gourmand,"Air conditioning,American Express credit card,...","Sit under vaulted, red-brick ceilings to enjoy..."
6777,Zazie,"ul. Józefa 34, Cracow, 32 056, Poland","Cracow, Poland",€,French,19.946949,50.051240,4.850041e+10,https://guide.michelin.com/en/lesser-poland/cr...,http://www.zaziebistro.pl/,Bib Gourmand,"Air conditioning,Booking essential - dinner,Cr...",You’ll find this lively bistro in a corner spo...
6778,Butchery & Wine,"ul. Żurawia 22, Warsaw, 00 515, Poland","Warsaw, Poland",€€,"Meats and Grills, Traditional Cuisine",21.015495,52.228581,4.822502e+10,https://guide.michelin.com/en/masovia/warsaw/r...,https://www.butcheryandwine.pl/,Bib Gourmand,"American Express credit card,Booking essential...",The name of this modern bistro says it all: st...
